In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.base import clone
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import sys
from scipy.stats import ttest_ind, zscore
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import copy
import json

# # Step 1: Gathering and shaping the data
The challenge in working with a “real world” dataset is that it has not been pre-packaged for analysis.  Some things that are essential to my analysis are spread all over the internet; other things have to be transformed into formats that are suitable for analysis.  This takes a lot of thinking.  The code structure may not be the most Pythonic, but it reflects my step-by-step thinking about how I needed the data to look.

## Creating the Player Position Dictionary

The heart of my analysis is aggregating the PI by position group per team so that I can compare the PI at one step below the total team performance for a particular PI. This data is very challenging to get, especially for past years, so rather than search for it programmatically, I collected it and put the csv file up to github for easy download. The code involves changing the order of the names to put last name first and deal with compound last names and middle initials.

In [23]:
# Here I need to import the new player data file that I have up on github.
# This covers player data from the 2012-2020 season.

url = 'https://raw.githubusercontent.com/michaelschlitzer/2019_Footy_Player_Dictionary/master/expanded_player_data.csv'
player_df = pd.read_csv(url)

player_df = player_df[['Player', 'Team', 'Position', 'DOB', 'Height', 'Weight']]
player_df = player_df.dropna(axis = 0)

In [24]:
player_df['Height'] = player_df['Height'].str.rstrip('cm').astype(int)
player_df['Weight'] = player_df['Weight'].str.rstrip('kg').astype(int)
player_df['DOB'] = pd.to_datetime(player_df['DOB'])

In [25]:
player_df.head()

,Player,Team,Position,DOB,Height,Weight
0,Alex Keath,Adelaide,Defender,1992-01-20,197,91
1,Alex Spina,Adelaide,Midfield,1995-06-27,181,75
2,Andrew McPherson,Adelaide,Defender,1999-06-20,186,79
3,Andy Otten,Adelaide,Defender,1989-05-15,192,90
4,Anthony Wilson,Adelaide,Forward,1992-08-02,179,70


In [26]:
fixed_name = []
for name in player_df['Player']:
    one = name.split()
    if len(one) > 2:
        one[1:] = [' '.join(one[1:])]
        fixed_name.append(one)
    else:
        fixed_name.append(one)

reversed_name = []
for name in fixed_name:
    reverso = [name[-1], name[0]]
    
    reversed_name.append(reverso)

player_df['RN'] = reversed_name
player_df['Reversed Name'] = player_df['RN'].apply(', '.join)
player_df['YOB'] = player_df['DOB'].dt.year

player_pos = player_df.drop(['RN'], axis = 1)
player_pos.head()

,Player,Team,Position,DOB,Height,Weight,Reversed Name,YOB
0,Alex Keath,Adelaide,Defender,1992-01-20,197,91,"Keath, Alex",1992
1,Alex Spina,Adelaide,Midfield,1995-06-27,181,75,"Spina, Alex",1995
2,Andrew McPherson,Adelaide,Defender,1999-06-20,186,79,"McPherson, Andrew",1999
3,Andy Otten,Adelaide,Defender,1989-05-15,192,90,"Otten, Andy",1989
4,Anthony Wilson,Adelaide,Forward,1992-08-02,179,70,"Wilson, Anthony",1992


# Game level stats by position group by team

In [29]:
pi_abbrevs = ['DI', 'KI', 'MK', 'HB', 'GL', 'BH', 'HO', 'TK', 'RB', 'IF', 'CL', 'CG', 'FF', 'FA', 'BR', 'CP', 'UP',
             'CM', 'MI', '1%', 'BO', 'GA', '%P']

PI = ['Disposals', 'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit Outs', 'Tackles', 'Rebounds', 'Inside 50s',
     'Clearances', 'Clangers', 'Frees', 'Frees Against', 'Brownlow Votes', 'Contested Possessions', 'Uncontested Possessions',
     'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces', 'Goal Assists', '% Played']

teams = ['Adelaide', 'Brisbane', 'Carlton', 'Collingwood', 'Essendon', 'Fremantle', 'Geelong', 'Gold Coast', 
         'Greater Western Sydney', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond',
        'St. Kilda', 'Sydney', 'West Coast', 'Western Bulldogs']

years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [30]:
teams = ['Adelaide', 'Brisbane', 'Carlton', 'Collingwood', 'Essendon', 'Fremantle', 'Geelong', 'Gold Coast', 
         'Greater Western Sydney', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond',
        'St. Kilda', 'Sydney', 'West Coast', 'Western Bulldogs']


# This takes a very long time to run.

team_urls = ['adelaide', 'brisbanel', 'carlton', 'collingwood', 'essendon', 'fremantle', 'geelong', 'goldcoast',
            'gws', 'hawthorn', 'melbourne', 'kangaroos', 'padelaide', 'richmond', 'stkilda', 'swans', 'westcoast',
            'bullldogs']

year_str = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

season_team_game_PI = []
for year in year_str:
    team_game_PI = []
    for team in team_urls:
        url = 'https://afltables.com/afl/stats/teams/'+team+'/'+year+'_gbg.html'
        PIGBG19 = pd.read_html(url)
        team_PI = []
        for PIX, pid in zip(PIGBG19, PI):
            PIX.drop(PIX.tail(2).index, inplace = True)
            
            string_section = PIX[pid].iloc[:,0]
            float_section = PIX[pid].iloc[:,1:].apply(pd.to_numeric, errors = 'coerce')
            
            float_section['Player'] = string_section
            float_section['Season'] = year
            float_section['Season'] = float_section['Season'].astype(float)
            first_col = float_section.pop('Player')
            float_section.insert(0, 'Player', first_col)
            
            PIX = float_section
            
            team_PI.append(PIX)
        team_game_PI.append(team_PI)
    season_team_game_PI.append(team_game_PI)

    
# This takes a very long time. It's a lot of data.  You can do it in Spark, but it's not any faster.
# At the end though you wind up with 8 Years, 18 teams, 23 stats

# 2020 creates a strange problem, since it is only 18 rounds. Therefore I am doing that separately and then I'll
# add blank Rounds and append 2020 into the big list.

In [31]:
# just for this time I already ran this

team_game_PI_20 = []
for team in team_urls:
    url = 'https://afltables.com/afl/stats/teams/'+team+'/2020_gbg.html'
    PIGBG20 = pd.read_html(url)
    team_PI = []
    for PI20, pid in zip(PIGBG20, PI):
        PI20.drop(PI20.tail(2).index, inplace = True)
        # Here, if it worked above, I just have to exclude the Player column from the replace
        PI20.iloc[:,1:] = PI20.iloc[:,1:].apply(pd.to_numeric, errors = 'coerce')

        string_section = PI20[pid].iloc[:,0]
        float_section = PI20[pid].iloc[:,1:].apply(pd.to_numeric, errors = 'coerce')
            
        float_section['Player'] = string_section
        float_section['Season'] = 2020
        float_section['Season'] = float_section['Season'].astype(float)
        first_col = float_section.pop('Player')
        float_section.insert(0, 'Player', first_col)
        
        PI20 = float_section
                
        team_PI.append(PI20)
    team_game_PI_20.append(team_PI)

In [32]:
for tgp in range(len(teams)):
    for pid, pidn in zip(PI, range(len(PI))):
        step1 = team_game_PI_20[tgp][pidn]
        step1.insert(18, 'R19', np.nan)
        step1.insert(19, 'R20', np.nan)
        step1.insert(20, 'R21', np.nan)
        step1.insert(21, 'R22', np.nan)
        step1.insert(22, 'R23', np.nan)

In [33]:
len(season_team_game_PI)

8

In [34]:
season_team_game_PI.append(team_game_PI_20)

In [35]:
len(season_team_game_PI)

9

In [36]:
for season in season_team_game_PI:

    for team in season:
            for PI in season[3]:
                PI.replace({'Brown, Callum': 'Brown, Callum M'}, inplace = True)

            for PI in season[7]:
                PI.replace({'Lynch, Tom': 'Lynch, Tom J'}, inplace = True)

            for PI in season[13]:
                PI.replace({'Lynch, Tom': 'Lynch, Tom J'}, inplace = True)

            for PI in season[8]:
                PI.replace({'Reid, Sam': 'Reid, Sam J'}, inplace = True)

            for PI in season[11]:
                PI.replace({'Thompson, Scott': 'Thompson, Scott D'}, inplace = True)

            for PI in season[17]:
                PI.replace({'Williams, Bailey': 'Williams, Bailey J'}, inplace = True)

            for PI in season[16]:
                PI.replace({'Kennedy, Josh': 'Kennedy, Josh J'}, inplace = True)

            for PI in season[15]:
                PI.replace({'Kennedy, Josh': 'Kennedy, Josh P'}, inplace = True)

In [37]:
pi_abbrevs = ['DI', 'KI', 'MK', 'HB', 'GL', 'BH', 'HO', 'TK', 'RB', 'IF', 'CL', 'CG', 'FF', 'FA', 'BR', 'CP', 'UP',
             'CM', 'MI', '1%', 'BO', 'GA', '%P']

PI = ['Disposals', 'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit Outs', 'Tackles', 'Rebounds', 'Inside 50s',
     'Clearances', 'Clangers', 'Frees', 'Frees Against', 'Brownlow Votes', 'Contested Possessions', 'Uncontested Possessions',
     'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces', 'Goal Assists', '% Played']

PI_dict = {k:v for (k,v) in zip(PI, pi_abbrevs)}


new_column_basis = ['DI', 'KI', 'MK', 'HB', 'GL', 'BH', 'HO', 'TK', 'RB', 'IF', 'CL', 'CG', 'FF', 'FA', 'CP', 'UP', 
                    'CM', 'MI', '1%', 'BO', 'GA', 'Height', 'Weight', 'Age']


pos_prefixes = ['D', 'F', 'M', 'R']
            
long_column_headers = []
for pos in pos_prefixes:
    for stat in new_column_basis:
        new_col_head = pos+'-'+stat
        long_column_headers.append(new_col_head)


In [77]:
# So, here the problem now is that 2020 is a multiindex, while 2012-2019 aren't - I've stripped it out.

teams = ['Adelaide', 'Brisbane', 'Carlton', 'Collingwood', 'Essendon', 'Fremantle', 'Geelong', 'Gold Coast', 
         'Greater Western Sydney', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond',
        'St. Kilda', 'Sydney', 'West Coast', 'Western Bulldogs']

years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

season_team_round_PI = []
for y, year in zip(years, range(len(years))):
    team_round_PI = []
    for t, team in zip(teams, range(len(teams))):
        round_PI = []

        columns = season_team_game_PI[year][team][0].columns
        rounds = [r for r in columns if r.startswith('R')]
        round_list = season_team_game_PI[year][team][0][rounds]

        for rnd in round_list:

            round_rnd = pd.DataFrame()
            round_rnd['Player'] = season_team_game_PI[year][team][0]['Player']
            round_rnd['Season'] = y
            round_rnd['Disposals'] = season_team_game_PI[year][team][0][rnd]
            round_rnd['Kicks'] = season_team_game_PI[year][team][1][rnd]
            round_rnd['Marks'] = season_team_game_PI[year][team][2][rnd]
            round_rnd['Handballs'] = season_team_game_PI[year][team][3][rnd]
            round_rnd['Goals'] = season_team_game_PI[year][team][4][rnd]
            round_rnd['Behinds'] = season_team_game_PI[year][team][5][rnd]
            round_rnd['Hit Outs'] = season_team_game_PI[year][team][6][rnd]
            round_rnd['Tackles'] = season_team_game_PI[year][team][7][rnd]
            round_rnd['Rebounds'] = season_team_game_PI[year][team][8][rnd]
            round_rnd['Inside 50s'] = season_team_game_PI[year][team][9][rnd]
            round_rnd['Clearances'] = season_team_game_PI[year][team][10][rnd]
            round_rnd['Clangers'] = season_team_game_PI[year][team][11][rnd]
            round_rnd['Frees'] = season_team_game_PI[year][team][12][rnd]
            round_rnd['Frees Against'] = season_team_game_PI[year][team][13][rnd]
            round_rnd['Brownlow Votes'] = season_team_game_PI[year][team][14][rnd]
            round_rnd['Contested Possessions'] = season_team_game_PI[year][team][15][rnd]
            round_rnd['Uncontested Possessions'] = season_team_game_PI[year][team][16][rnd]
            round_rnd['Contested Marks'] = season_team_game_PI[year][team][17][rnd]
            round_rnd['Marks Inside 50'] = season_team_game_PI[year][team][18][rnd]
            round_rnd['One Percenters'] = season_team_game_PI[year][team][19][rnd]
            round_rnd['Bounces'] = season_team_game_PI[year][team][20][rnd]
            round_rnd['Goal Assists'] = season_team_game_PI[year][team][21][rnd]
            round_rnd['% Played'] = season_team_game_PI[year][team][22][rnd]
            
            merged_round_rnd = pd.merge(round_rnd, player_pos, how = 'left', left_on = 'Player', right_on = 'Reversed Name')
            merged_round_rnd = merged_round_rnd.drop(columns = ['Player_y'], axis = 1)
            merged_round_rnd.rename(columns = {'Player_x': 'Player'}, inplace = True)
            merged_round_rnd['Season'] = merged_round_rnd['Season'].astype(int)
            merged_round_rnd['Age'] = merged_round_rnd['Season'] - merged_round_rnd['YOB']
            merged_round_rnd.drop(labels = ['YOB'], axis = 1, inplace = True)
            
            # Here is where I re-arrange and reduce the number of PI.
            
            merged_round_rnd = merged_round_rnd[['Player', 'Team', 'Season', 'Position', 'Disposals', 'Kicks', 'Marks', 
                                                 'Handballs','Goals', 'Behinds', 'Hit Outs', 'Tackles', 'Rebounds', 
                                                 'Inside 50s', 'Clearances', 'Clangers', 'Frees', 'Frees Against', 
                                                 'Contested Possessions', 'Uncontested Possessions', 'Contested Marks',
                                                 'Marks Inside 50', 'One Percenters', 'Bounces', 'Goal Assists',
                                                 'Height', 'Weight', 'Age']]
            
            merged_round_rnd.rename(columns = PI_dict, inplace = True)
            
            ###
            
            for row in range(len(merged_round_rnd)):
                if merged_round_rnd.iloc[row,4:25].sum() == 0:
                    merged_round_rnd.iloc[row,25] = np.nan
                    merged_round_rnd.iloc[row,26] = np.nan
                    merged_round_rnd.iloc[row,27] = np.nan
                else:
                    pass
                    
            r_sums = merged_round_rnd[['Position','DI', 'KI', 'MK', 'HB', 'GL','BH', 'HO', 'TK', 'RB', 'IF',
                                       'CL', 'CG', 'FF', 'FA', 'CP', 'UP', 'CM','MI', '1%', 'BO', 'GA']]
            
            r_means = merged_round_rnd[['Position', 'Height', 'Weight', 'Age']]

            r_sigma = r_sums.groupby(['Position']).sum()
            
            r_sums_max = r_sums.groupby(['Position']).max()
            r_sums_min = r_sums.groupby(['Position']).min()
            r_sums_mean = r_sums.groupby(['Position']).mean()
            r_sums_std = r_sums.groupby(['Position']).std()
            
            r_mu = r_means.groupby(['Position']).mean()
            
            r_means_max = r_means.groupby(['Position']).max()
            r_means_min = r_means.groupby(['Position']).min()
            r_means_mean = r_means.groupby(['Position']).mean()
            r_means_std = r_means.groupby(['Position']).std()
            
            r_together = pd.merge(r_sigma, r_mu, how = 'left', left_index = True, right_index = True)
            
            defender = r_together.loc['Defender']
            forward = r_together.loc['Forward']
            midfield = r_together.loc['Midfield']
            ruck = r_together.loc['Ruck']
            
            round_team_concat = pd.concat([defender, forward, midfield, ruck])
            
            rtc_df = pd.DataFrame(round_team_concat).T
            rtc_df.columns = long_column_headers
            
            rtc_df.insert(0, 'Team', t)
            rtc_df.insert(1, 'Season', str(y))
            rtc_df.insert(2, 'Round', rnd)
            rtc_df.set_index('Round', inplace = True)
            
            rtc_dict = rtc_df.to_dict()
            
            round_PI.append(rtc_dict)
            # round_PI.append(rtc_df) # I've undone this in order to be able to save it as a json file.
        team_round_PI.append(round_PI)
    season_team_round_PI.append(team_round_PI)

In [78]:
outpath = 'AFL_Season_Team_PI_Data.json'
with open(outpath, 'w') as f:
    f.write(json.dumps(season_team_round_PI))

In [ ]:
# Now, at this point I am writing this JSON file to my local drive and need to move it manually to GitHub.  I want this
# to be manual to prevent any useless override of existing data.